## Imports

In [75]:
from glob import glob
from openpyxl import load_workbook
from random import randint
from google.colab import files

## Configuration

In [76]:
#mode = "lines"
#mode = "phrases"
mode = "sentences"

In [124]:
if mode == "phrases":
  match_lines = False
  word_split_token = " "
  characters_to_replace_with_word_split_token = ",!?\n\r\\"

In [146]:
if mode == "sentences":
  match_lines = False
  word_split_token = "."
  characters_to_replace_with_word_split_token = "\r\n\\"

In [126]:
if mode == "lines":
  match_lines = True
  word_split_token = "."
  characters_to_replace_with_word_split_token = "\n\r\\"

In [127]:
min_phrase_length = 3

# File Upload

In [81]:
input_files = files.upload()

Saving Carmilla - Joseph Sheridan le Fanu.txt to Carmilla - Joseph Sheridan le Fanu (2).txt


In [82]:
input_files.keys()

dict_keys(['Carmilla - Joseph Sheridan le Fanu.txt'])

## Book Chopping

In [83]:
def num_words(token, a=2, b=7):
  if token == ".":
    return 1
  else:
    return randint(a, b) + randint(a, b)

In [84]:
def clean(phrase: str):
  phrase = phrase.replace("â€", ",")
  for char in characters_to_replace_with_word_split_token:
    phrase = phrase.replace(char, word_split_token)
  return phrase.strip()

In [85]:
def get_words(input_files:list, word_split_token:str):
  words = []
  for book in input_files.values():
    words.extend(clean(str(book)).split(word_split_token))
  return words

In [107]:
def get_phrases_random(input_files, word_split_token):
  phrases = []
  words = get_words(input_files, word_split_token)
  while True:
    try:
      n = num_words(word_split_token)
      phrase = [words.pop(0) for _ in range(n)]
    except IndexError:
      phrase = words
      break
    finally:
      phrase = " ".join(phrase).strip()
      if len(phrase) >= min_phrase_length:
        phrases.append(phrase)
  return phrases

In [108]:
def get_phrases_lines(input_files):
  lines = []
  for book in input_files.values():
    book = str(book)
    for line in book:
      line = clean(line)
      if len(line) >= min_phrase_length:
        lines.append(line)
  return lines

In [88]:
def get_phrases(input_files, match_lines=False, word_split_token=" "):
  if match_lines:
    return get_phrases_lines(input_files)
  else:
    return get_phrases_random(input_files, word_split_token)

In [147]:
phrases = get_phrases(input_files, match_lines, word_split_token)

In [150]:
phrases[916]

'My sufferings had, during the last week, told upon my appearance'

## File Writing

wb = load_workbook(filename="template.xlsx")
phrases_sheet = wb["phrases"]
for i, phrase in enumerate(phrases):
    phrases_sheet.append((i, phrase))
wb.save(output_file_name)